# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
sys.path.append('../../src')

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies


Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding_same-5556000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo_same-5556000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/Jack/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/dummy_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/righty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/decl_nt-950000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/dummy_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/righty_suit-

### Bidding

In [2]:
vuln_ns, vuln_ew = False, True
hand = 'A.8xx.Jxx.AQxxxx'

auction = ["1S"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 1, 0, False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?


[{'call': 'PASS',
  'insta_score': 0.257,
  'expected_score': -2,
  'adjustment': 13},
 {'call': '2C', 'insta_score': 0.741, 'expected_score': -28, 'adjustment': 37}]

In [3]:
auction = ["1S","PASS","1N","PASS","2C"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 1, 0, False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']


[{'call': 'PASS',
  'insta_score': 0.843,
  'expected_score': 22,
  'adjustment': 42},
 {'call': 'X', 'insta_score': 0.108, 'expected_score': 2, 'adjustment': 5}]

In [4]:
### Samples consistent with the auction so far

for i in range(min(10,len(bid.samples))):
    print(bid.samples[i])


KJ8x.x.Qx.KJ98xx A.8xx.Jxx.AQxxxx xx.KQ9xx.KT98xx. QT9xxx.AJTx.Ax.T 0.74421 
 1S-P-1N-P-2C-P-2D-P-3C-P-P-P (4.9, 5.1)  
 1S-P-1N-P-2C-X-2D-P-3C-P-3D-P-3H-P-P-P (4.8, 6.7) 
8xxx.AJ.ATx.KT8x A.8xx.Jxx.AQxxxx xx.KQ9x.K9xxx.Jx KQJT9x.Txxx.Q8.9 0.74986 
 1S-P-1N-P-2C-P-2S-P-P-P (4.9, 6.0)  
 1S-P-1N-P-2C-X-P-P-P (6.2, 7.2) 
QT98xx.Ax.K.JT9x A.8xx.Jxx.AQxxxx x.KJTxx.QT98xx.K KJxxx.Q9x.Axx.8x 0.74532 
 1S-P-1N-P-2C-P-2D-P-2S-P-P-P (7.2, 6.6)  
 1S-P-1N-P-2C-X-2D-P-2S-P-P-P (7.2, 6.6) 
QTxxx.A.AKx.K98x A.8xx.Jxx.AQxxxx Kxx.QTxx.Q98xx.J J98x.KJ9xx.Tx.Tx 0.74781 
 1S-P-1N-P-2C-P-2S-P-P-P (9.7, 9.0)  
 1S-P-1N-P-2C-X-P-2H-P-P-2S-3H-P-P-3S-P-P-P (9.7, 9.0) 
KQ98xx.K9.AK.Kxx A.8xx.Jxx.AQxxxx x.AQTxxx.Q9xx.J9 JTxxx.Jx.T8xx.T8 0.74822 
 1S-P-1N-P-2C-P-2H-P-3S-P-P-P (9.2, 9.0)  
 1S-P-1N-P-2C-X-2H-P-3S-P-P-P (9.2, 9.0) 
QJTx.Kx.9x.KJ8xx A.8xx.Jxx.AQxxxx Kx.QJ9xx.KT8x.T9 98xxxx.ATx.AQxx. 0.74962 
 1S-P-1N-P-2C-P-2S-P-4D-P-P-P (3.8, 6.1)  
 1S-P-1N-P-2C-X-P-2D-3C-P-P-P (5.1, 5.5) 
KJ9xxx.x.K8x.J9x A.8xx

West always has the same cards because it's your hand and it's known to you

In [5]:
# we got some samples above. if we want more, we can always get more
auction = ["PASS", "1N"]
sample_hands, sorted_score, p_hcp, p_shp, good_quality = bot_bid.sample_hands_for_auction(auction,2)
print("Good quality:", good_quality)
for i in range(min(10,sample_hands.shape[0])):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])} {sorted_score[i]:.2f}')

Good quality: True
xxx.QTx.T98x.KJT QJT.AKx.AKxx.98x A.8xx.Jxx.AQxxxx K98xxx.J9xx.Qx.x 1.00
QTx.QT9x.xxx.KTx KJxxx.KJx.AKT.9x A.8xx.Jxx.AQxxxx 98xx.Axx.Q98x.J8 0.89
QT8xx.K9xx.xx.8x KJx.JTxx.K9xx.Kx A.8xx.Jxx.AQxxxx 9xxx.AQ.AQT8.JT9 0.75
QJTxxx.JTxx.Ax.9 Kxx.Q9xx.Q9.KJTx A.8xx.Jxx.AQxxxx 98x.AK.KT8xxx.8x 0.71
98xx.Kxxx.8x.T9x QJxx.AQT.KQxx.Jx A.8xx.Jxx.AQxxxx KTxx.J9x.AT9x.K8 1.00
Q9xx.KQx.K9xx.xx Kxx.AJ9xx.QT8.KJ A.8xx.Jxx.AQxxxx JT8xx.Tx.Axx.T98 0.75
9xx.K9xxx.Q9x.Tx Qxxxx.AJT.ATx.J8 A.8xx.Jxx.AQxxxx KJT8.Qx.K8xx.K9x 0.75
QJT.KJxx.QTx.9xx xxx.AQx.K98x.KJ8 A.8xx.Jxx.AQxxxx K98xxx.T9x.Axx.T 0.75
K9xx.xx.9xxxx.9x J8xx.KQxx.AK8.8x A.8xx.Jxx.AQxxxx QTxx.AJT9.QT.KJT 0.75
T8xxx.QJxx.A9x.x QJ9.AKx.KQxx.K9x A.8xx.Jxx.AQxxxx Kxxx.T9x.T8x.JT8 0.75
